In [1]:
from vpython import *


def Lorentz_F(q,v,E,B):
    F = q*(E+cross(v,B))
    return F
def update_v(vi,m,F,dt):
    vf = vi + F*dt/m
    return vf
def update_position(ri,vi,m,F,dt):
    rf = ri + vi*dt + 0.5*F*dt*dt/m
    return rf

v = vector(1,0,0)
E = vector(5,0,0)
B = vector(0,0,2)
q = 1e-9
Lorentz_F(q,v,E,B)



<IPython.core.display.Javascript object>

<5e-09, -2e-09, 0>

In [6]:
scene = canvas() 
######################################################
#define parameters for the moving point charge
obj = sphere(radius = 1) 
obj.charge = -1
obj.mass = 1 
obj.pos = vector(0,0,0) #initial position vector
obj.velocity = vector(4,0,0) #initial velocity vector
obj.color = color.blue
if obj.charge >0:
    obj.color = color.red
######################################################
#define the uniform E and B fields 
E = vector(0,4,0)
B = vector(0,0,2)
######################################################
t = 0 #initial time
dt = 0.001 #timestep to evolve the system
trail = curve(color=color.white)
max_y = 15
max_x = 50
if E.y != 0:
    mybox = box(pos=vector(0,max_y*E.y/abs(E.y),0), length=max_x, height=0.1, width=50,color=color.blue)
    mybox = box(pos=vector(0,-max_y*E.y/abs(E.y),0), length=max_x, height=0.1, width=50,color=color.red)
    dx = max_x/10
    for x in arange(-max_x/2, max_x/2+dx, 2*dx):
        arrow(pos=vector(x,-max_y*E.y/abs(E.y),0),axis = vector(0,2*max_y*E.y/abs(E.y),0),color=vector(0,.8,.8),shaftwidth=0.1)

if B.z != 0:
    
    dx = max_x/10
    for x in arange(-max_x/2, max_x/2+dx, 2*dx):
        arrow(pos=vector(x,0,0),axis = vector(0,0,2*max_y*B.z/abs(B.z)),color=color.yellow,shaftwidth=0.3)
    #scene.center = vector(max_x/3,0,0)


scene.range = max_y*2*1.1
while t<20:
    
    rate(600) #slow down the rate of visualization to 600 updates per sec
    F = Lorentz_F(obj.charge,obj.velocity,E,B) #calculate the net force on the charge
    r = update_position(obj.pos,obj.velocity,obj.mass,F,dt)
    v = update_v(obj.velocity,obj.mass,F,dt)
    obj.pos = r
    obj.velocity = v
    trail.append(pos=r)
    t = t+dt
    if abs(r.y) >=max_y:
        break
    

<IPython.core.display.Javascript object>

KeyboardInterrupt: 

In [60]:
scene = canvas() 
trail = curve(color=color.yellow)
myell = ellipsoid(pos=vector(0,0,0),length=1, height=1, width=0.1,color=color.white,opacity = 0.3)
box(pos=vector(0,0,0), length=0.2, height=1, width=0.1,color=color.black,opacity = 0.8)
scene.lights = []
distant_light(direction=vec( 1.1,  0.44,  0.88), color=color.gray(0.8))
obj = sphere(radius = 0.06) 
obj.charge = 1
obj.mass = 1 
obj.pos = vector(0,0,0) #initial position vector
obj.velocity = vector(0,0,0) #initial velocity vector
obj.color = color.blue
if obj.charge >0:
    obj.color = color.red
    

######################################################
t = 0 #initial time
dt = 0.001 #timestep to evolve the system
in_B = False
n = 1
while obj.pos.mag <0.5:
    
    rate(600) #slow down the rate of visualization to 600 updates per sec
    if abs(obj.pos.x) >=0.1:
        E = vector(0,0,0)
        B = vector(0,0,4)
        in_B = True
    else:
        if in_B:
            in_B = False
            n = -n
        E = vector(0.3*n,0,0)
        B = vector(0,0,0)
        
    F = Lorentz_F(obj.charge,obj.velocity,E,B) #calculate the net force on the charge
    r = update_position(obj.pos,obj.velocity,obj.mass,F,dt)
    v = update_v(obj.velocity,obj.mass,F,dt)
    obj.pos = r
    obj.velocity = v
    trail.append(pos=r)
    t = t+dt
    
    

<IPython.core.display.Javascript object>